## En este notebook vamos ver cómo aplicar los diferentes algoritmos de clustering a la data de iris y algunas aplicaciones de la PCA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt

Esta dataset se podía cargar directamente de scikit, dentro del módulo datasets

In [ ]:
from sklearn.datasets import load_iris

In [ ]:
#Cargo los datos
data = load_iris()

In [ ]:
#Defino la matriz de features
X = data.data

In [ ]:
#Defino el vector de targets
y = data.target

In [ ]:
#Miro la distribución de las clases para cada variable mediante histogramas

fig, axs= plt.subplots(nrows=2, ncols=2)
axs = axs.flatten()

for i in range(X.shape[1]):
    
    axs[i].hist(X[y==0,i])
    axs[i].hist(X[y==1,i])
    axs[i].hist(X[y==2,i])

In [ ]:
# También usando scatter plots
fig, axs= plt.subplots(nrows=3, ncols=2)
axs = axs.flatten()

axs[0].scatter(X[:,0],X[:,1], c = y)
axs[1].scatter(X[:,0],X[:,2], c = y)
axs[2].scatter(X[:,0],X[:,3], c = y)
axs[3].scatter(X[:,1],X[:,2], c = y)
axs[4].scatter(X[:,1],X[:,3], c = y)
axs[5].scatter(X[:,2],X[:,3], c = y)

Vamos a estandarizar los datos, puesto que lo vamos a necesitar para hacer el kmeans (dado que todas las variables tienen que ser del mismo rango) y para hacer la pca

In [ ]:
from sklearn import preprocessing

ss = preprocessing.StandardScaler()

X_scaled = pd.DataFrame( ss.fit_transform(X), columns = data.feature_names)

Probamos diferentes algoritmos de clustering y vemos su rendimiento en la reconstrucción de los tres grupos

In [ ]:
from sklearn.cluster import KMeans

nclusters = 3

km = KMeans(n_clusters=nclusters, random_state=0, n_init=100)
km.fit(X_scaled)

# predict the cluster for each data point
y_cluster_kmeans = km.predict(X_scaled)
y_cluster_kmeans

In [ ]:
from sklearn.mixture import GaussianMixture

gmm = GaussianMixture(n_components=nclusters, n_init = 100, random_state=100)
gmm.fit(X_scaled)

# predict the cluster for each data point
y_cluster_gmm = gmm.predict(X_scaled)
y_cluster_gmm

In [ ]:
from sklearn.cluster import AgglomerativeClustering

agg = AgglomerativeClustering(n_clusters=3,compute_full_tree=True)
y_cluster_agg = agg.fit_predict(X_scaled)
y_cluster_agg

In [ ]:
from sklearn.metrics import completeness_score
print(completeness_score(y, y_cluster_kmeans))
print(completeness_score(y, y_cluster_gmm))
print(completeness_score(y, y_cluster_agg))

In [ ]:
from sklearn.metrics import adjusted_mutual_info_score
print(adjusted_mutual_info_score(y, y_cluster_kmeans))
print(adjusted_mutual_info_score(y, y_cluster_gmm))
print(adjusted_mutual_info_score(y, y_cluster_agg))

Veamos el número de clusters usando el método elbow. En este caso, podemos llamar al atributo *inertia_* de kmeans después de fittearlo. Dicho atributo nos devuelve la suma del cuadrado de las distancias a los centroides, que es lo que optimiza kmeans. Para más info, podéis ver la documentación de kmeans en scikit http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html

In [ ]:
kmeans_scores = []
for k in np.arange(1,20):
    km = KMeans(n_clusters=k, random_state=0, n_init=100)
    km.fit(X_scaled)
    kmeans_scores.append(km.inertia_)

plt.plot(np.arange(1,20), kmeans_scores)
plt.xticks(np.arange(1,20))
pass

### PCA para la visualización

Vamos a ver cómo usar la pca, en este caso para facilitarnos la visualización de las clases

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)

X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(data = X_pca
             , columns = ['pc1', 'pc2'])

In [ ]:
plt.scatter(pca_df.iloc[y==0,0].values,pca_df.iloc[y==0,1].values)
plt.scatter(pca_df.iloc[y==1,0].values,pca_df.iloc[y==1,1].values)
plt.scatter(pca_df.iloc[y==2,0].values,pca_df.iloc[y==2,1].values)
plt.legend(data.target_names)

Una vez fitteada la pca, podemos ver el ratio de la variación explicada por cada componente accediendo al atributo *explained_variance_ratio_* del objeto de la pca. Para más info, podéis acceder a la documentación de la clase PCA en scikit http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html

In [ ]:
print( "la variacion de cada compenentes es = " , pca.explained_variance_ratio_)
print( "la variacion total explicada por las componentes de la pca es = " , np.sum(pca.explained_variance_ratio_))

In [ ]:
pca.explained_variance_ratio_

In [ ]:
plt.bar([1,2], pca.explained_variance_ratio_)
plt.xticks([1,2])
pass

También podríamos haberle pedido que nos hiciera la descomposición en tres componentes

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=3)

X_pca = pca.fit_transform(X_scaled)

pca_df = pd.DataFrame(data = X_pca
             , columns = ['pc1', 'pc2','pc3'])

In [ ]:
print( "la variacion total explicada en este caso es = " , np.sum(pca.explained_variance_ratio_))

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(pca_df.iloc[y==0,0].values,pca_df.iloc[y==0,1].values, pca_df.iloc[y==0,2].values)
ax.scatter(pca_df.iloc[y==1,0].values,pca_df.iloc[y==1,1].values, pca_df.iloc[y==0,2].values)
ax.scatter(pca_df.iloc[y==2,0].values,pca_df.iloc[y==2,1].values,pca_df.iloc[y==0,2].values)
ax.legend(data.target_names)

Por otro lado, cuando usamos la pca, también podemos pasarle la variación de los datos que queremos que explique. Se lo pasamos como un float entre 0 y 1 al argumento *n_components*. De esta manera, la pca coge las componentes que calculan la variación de los datos que queremos explicar. Una vez ajustada la pca a los datos, podemos acceder al atributo *n_components_* (con una sola barra baja al final) para saber las componentes seleccionadas

In [ ]:
# Por ejemplo pedimos al menos el 70%
pca = PCA(n_components=0.70)
pca.fit(X_scaled)
print( "El numero de componentes seleccionadas es = " , pca.n_components_)
print( "la variacion total explicada en este caso es = " , np.sum(pca.explained_variance_ratio_))

### PCA para reducir el tiempo computacional (Opcional)

Podemos usar pca para reducir el número de dimensiones y así reducir la carga computacional

In [ ]:
from sklearn.datasets import fetch_mldata

mnist = fetch_mldata('MNIST original')

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(mnist.data, 
                                                    mnist.target, 
                                                    test_size=0.2, random_state=0)

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

X_train = ss.fit_transform(X_train)
X_test = ss.transform(X_test)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(.95)

In [ ]:
pca.fit(X_train)

In [ ]:
X_train_pca = pca.transform(X_train)
X_test_pca = pca.transform(X_test)

In [ ]:
from sklearn.linear_model import SGDClassifier
clf = SGDClassifier(random_state=0)

In [ ]:
import time

start_time = time.time()
clf.fit(X_train, y_train)
end_time = time.time()
print("el tiempo que ha tardado usando toda la data:", end_time - start_time)
print(clf.score(X_test, y_test))

In [ ]:
start_time = time.time()
clf.fit(X_train_pca, y_train)
end_time = time.time()
print("el tiempo que ha tardado usando el 95%:", end_time - start_time)
print(clf.score(X_test_pca, y_test))